In [ ]:
import rospy
from geometry_msgs.msg import PoseStamped
from nav_msgs.msg import Odometry
from my_assignment_2.msg import PosVel

import actionlib # Bring in the SimpleActionClient

# Bring in the PlanningAction type, composed by the types PlanningGoal, PlanningResult and PlanningFeedback
from assignment_2_2022.msg import PlanningAction, PlanningGoal

# Since we'll publish within the callback function, we have to define the publisher globally
pub = rospy.Publisher("/pos_vel", PosVel, queue_size=10)

In [ ]:
def callback_odometry(odometry):
    
    my_pos_vel = PosVel() # this is the msg we'll publish
    
    # Fill the four fields relying on the data received via the topic /odom
    my_pos_vel.x = odometry.pose.pose.position.x
    my_pos_vel.y = odometry.pose.pose.position.y
    my_pos_vel.vel_x = odometry.twist.twist.linear.x
    my_pos_vel.vel_z = odometry.twist.twist.angular.z

    pub.publish(my_pos_vel) # publish

In [ ]:
# Initialize a rospy node so that the SimpleActionClient can publish and subscribe over ROS
rospy.init_node('planning_client')

# Subscribe to the topic /odom
rospy.Subscriber("/odom", Odometry, callback_odometry)

# Counters for the number of goals reached and cancelled, in total
n_goals_reached = 0
n_goals_cancelled = 0

# Initialize the parameters, in the ROS parameter service, related to these counters
rospy.set_param('goals_reached', n_goals_reached)
rospy.set_param('goals_cancelled', n_goals_cancelled)

# Create the SimpleActionClient. The service is called '/reaching_goal' and the type of action is 'PlanningAction'
client = actionlib.SimpleActionClient('reaching_goal', PlanningAction)

# Wait until the action server has started up and started listening for goals
print("Waiting for the action server to start...")
client.wait_for_server()
print("Action server up and running!\n")

In [ ]:
import ipywidgets as widgets

# Get a new target from user input

print("Please, enter a new target and click \'Send\':\n")
print("x = ")
x = widgets.FloatText()
display(x)
print("y = ")
y = widgets.FloatText()
display(y)

button = widgets.Button(
    description='Send',
    disabled=False,
    button_style='success'
)

# cancel_button = widgets.Button(
#     description='Cancel',
#     disabled=False,
#     button_style='danger'
# )

output = widgets.Output()
display(button, output)

def on_button_clicked(b):
    with output:
        
        # Create a goal to send to the action server
        my_goal_pose = PoseStamped()
        my_goal_pose.pose.position.x = x.value
        my_goal_pose.pose.position.y = y.value
        goal = PlanningGoal(target_pose = my_goal_pose)

        # Send the goal to the action server
        client.send_goal(goal)
        
        print("Goal sent to the action server")
        print("Heading to the goal... Click \'Cancel\' to cancel the goal")
        
#         display(cancel_button, output)
        
button.on_click(on_button_clicked)


In [ ]:
# def on_cancel_button_clicked(b):
#     with output:
        
#         client.cancel_goal()
#         print("Goal cancelled\n")
#         n_goals_cancelled += 1
#         rospy.set_param('goals_cancelled', n_goals_cancelled)
#         return True

# while (1):
#     # If the goal has been successfully reached (sratus=3), update the parameter related to the number of goals
#     # reached and break the loop
#     if client.get_state() == 3:
#         print("Goal reached!\n")
#         n_goals_reached += 1
#         rospy.set_param('goals_reached', n_goals_reached)
#         break
    
#     # If the user clicks the 'cancel' button, cancel the current goal, update the parameter related to the number 
# #     of goals cancelled and break the loop
#     if cancel_button.on_click(on_cancel_button_clicked):
#         break